In [ ]:
import h5py as h5
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import csv
import h5py
from torchvision.models import resnet18
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import warnings 
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
train_path = "/kaggle/input/deeplense-task1/dataset/train"
train_imgs = {}
for cat in os.listdir(train_path):
    sub_path = os.path.join(train_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    train_imgs[cat] = imgs

In [3]:
val_path = "/kaggle/input/deeplense-task1/dataset/val"
val_imgs = {}
for cat in os.listdir(val_path):
    sub_path = os.path.join(val_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    val_imgs[cat] = imgs

In [4]:
train_rows = [(img_path, label) for label, img_paths in train_imgs.items() for img_path in img_paths]
val_rows = [(img_path, label) for label, img_paths in val_imgs.items() for img_path in img_paths]

In [5]:
df_train = pd.DataFrame(train_rows, columns = ['img_path','label'])
df_val = pd.DataFrame(val_rows, columns = ['img_path','label'])

In [6]:
df_train['label'] = df_train['label'].map({'no':0, 'sphere':1, 'vort':2})
df_val['label'] = df_val['label'].map({'no':0, 'sphere':1, 'vort':2})

In [7]:
class Dataset(nn.Module):
    def __init__(self,data,transform = None, train = True):
        self.data = data
        self.imgs = [np.load(img_path) for img_path in self.data['img_path']]
        self.y = torch.tensor(self.data['label'], dtype = torch.long)
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.imgs[idx].squeeze()[:,:,np.newaxis]
        label = self.y[idx]
        img = np.tile(img,(1,1,3))
        if self.transform != None:
            img = self.transform(image = img)["image"]

        return img, label

In [8]:
train_transform = A.Compose([
    # A.RandomRotate90(p=0.5),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    # A.Normalize(mean=(0.485, 0.456, 0.406), 
    #             std=(0.229, 0.224, 0.225)),
    # A.Resize(28,28),
    ToTensorV2()
])

val_transform = A.Compose([
    # A.Resize(28,28),
    ToTensorV2()
])

In [9]:
train_dataset = Dataset(df_train, train_transform)
val_dataset = Dataset(df_val, val_transform)

In [10]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [ ]:
class FourierAuxCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(FourierAuxCNN, self).__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 37 * 37, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        self.fourier_head = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 2, kernel_size=3, padding=1)
        )
    
    def forward(self, x):
        features = self.shared(x)
        class_out = self.classifier(features)
        fourier_pred = self.fourier_head(features)
        return class_out, fourier_pred

def fourier_loss(predicted, x):
    x_gray = x.mean(dim=1, keepdim=True)
    x_fft = fft.fft2(x_gray) 
    pred_complex = torch.complex(predicted[:,0:1], predicted[:,1:2])
    loss = F.mse_loss(torch.abs(pred_complex), torch.abs(x_fft))
    return loss

def get_pretrained_model(model_name, num_classes=3, pretrained=True):
    if model_name.lower() == 'fourierauxcnn':
        model = FourierAuxCNN(num_classes=num_classes)
        return model
    else:
        try:
            model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)
            return model
        except Exception as e:
            raise ValueError(f"Error creating model '{model_name}' with timm: {e}")

def train_fourier_aux(model_name, train_dataloader, val_dataloader, num_epochs=10, device='cuda', lambda_fourier=0.1):
    model = get_pretrained_model(model_name, num_classes=3, pretrained=True)
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    is_aux_model = hasattr(model, 'fourier_head')
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        
        epoch_true = []
        epoch_preds = []
        
        for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            images = images.float().to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            if is_aux_model:
                outputs, fourier_pred = model(images)
                cls_loss = criterion(outputs, labels)
                aux_loss = fourier_loss(fourier_pred, images)
                loss = cls_loss + lambda_fourier * aux_loss
                preds = F.softmax(outputs, dim=1)
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                preds = F.softmax(outputs, dim=1)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * images.size(0)
            total += labels.size(0)
            _, predicted = (outputs.max(1) if is_aux_model else torch.max(outputs, 1))
            correct += predicted.eq(labels).sum().item()
            
            epoch_true.extend(labels.cpu().numpy())
            epoch_preds.extend(preds.cpu().detach().numpy())

            del images, labels, outputs, preds
            if is_aux_model:
                del fourier_pred, cls_loss, aux_loss
        
        try:
            auc = roc_auc_score(epoch_true, epoch_preds, multi_class='ovr')
        except Exception as e:
            auc = float('nan')
            print("AUC calculation error:", e)
        
        print(f"Epoch {epoch+1}: Train Loss {total_loss/total:.4f}, Train Acc {correct/total:.4f}, Train AUC {auc:.4f}")
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        total_val = 0
        val_true = []
        val_preds = []
        with torch.no_grad():
            for images, labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                images = images.float().to(device)
                labels = labels.to(device)
                if is_aux_model:
                    outputs, fourier_pred = model(images)
                    cls_loss = criterion(outputs, labels)
                    aux_loss = fourier_loss(fourier_pred, images)
                    loss = cls_loss + lambda_fourier * aux_loss
                    preds = F.softmax(outputs, dim=1)
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    preds = F.softmax(outputs, dim=1)
                val_loss += loss.item() * images.size(0)
                total_val += labels.size(0)
                _, predicted = (outputs.max(1) if is_aux_model else torch.max(outputs, 1))
                val_correct += predicted.eq(labels).sum().item()
                
                val_true.extend(labels.cpu().numpy())
                val_preds.extend(preds.cpu().detach().numpy())
                
                del images, labels, outputs, preds
                if is_aux_model:
                    del fourier_pred, cls_loss, aux_loss
                
        try:
            val_auc = roc_auc_score(val_true, val_preds, multi_class='ovr')
        except Exception as e:
            val_auc = float('nan')
            print("Validation AUC calculation error:", e)
        
        print(f"Epoch {epoch+1}: Val Loss {val_loss/total_val:.4f}, Val Acc {val_correct/total_val:.4f}, Val AUC {val_auc:.4f}")
    return model

In [12]:
train_fourier_aux('efficientnet_b0', train_loader, val_loader)

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1/10 [Train]: 100%|██████████| 938/938 [01:12<00:00, 12.99it/s]


Epoch 1: Train Loss 1.1433, Train Acc 0.4664, Train AUC 0.6554


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:06<00:00, 38.91it/s]


Epoch 1: Val Loss 0.9016, Val Acc 0.5647, Val AUC 0.8095


...

Epoch 9: Val Loss 0.2931, Val Acc 0.8920, Val AUC 0.9806


Epoch 10/10 [Train]: 100%|██████████| 938/938 [01:10<00:00, 13.28it/s]


Epoch 10: Train Loss 0.1540, Train Acc 0.9446, Train AUC 0.9922


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:05<00:00, 42.80it/s]

Epoch 10: Val Loss 0.2190, Val Acc 0.9216, Val AUC 0.9844


In [16]:
model = train_fourier_aux('efficientnet_b1', train_loader, val_loader)

Epoch 1/10 [Train]: 100%|██████████| 938/938 [01:37<00:00,  9.62it/s]


Epoch 1: Train Loss 1.1820, Train Acc 0.3528, Train AUC 0.5231


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:07<00:00, 32.65it/s]


Epoch 1: Val Loss 1.0684, Val Acc 0.3767, Val AUC 0.5901


Epoch 2/10 [Train]: 100%|██████████| 938/938 [01:37<00:00,  9.59it/s]


...

Epoch 9: Val Loss 0.2291, Val Acc 0.9223, Val AUC 0.9835


Epoch 10/10 [Train]: 100%|██████████| 938/938 [01:37<00:00,  9.60it/s]


Epoch 10: Train Loss 0.1909, Train Acc 0.9331, Train AUC 0.9881


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:07<00:00, 32.80it/s]

Epoch 10: Val Loss 0.2690, Val Acc 0.9037, Val AUC 0.9807


In [17]:
torch.save({'model_state_dict':model.state_dict()},'effnet_b1_fourier_loss.pth')

In [13]:
model = train_fourier_aux('resnet18', train_loader, val_loader)

Epoch 1/10 [Train]: 100%|██████████| 938/938 [00:39<00:00, 23.55it/s]


Epoch 1: Train Loss 0.9759, Train Acc 0.4717, Train AUC 0.6783


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 53.73it/s]


Epoch 1: Val Loss 1.2352, Val Acc 0.4991, Val AUC 0.7834


Epoch 2/10 [Train]: 100%|██████████| 938/938 [00:39<00:00, 23.62it/s]


...

Epoch 9: Val Loss 0.3315, Val Acc 0.8896, Val AUC 0.9806


Epoch 10/10 [Train]: 100%|██████████| 938/938 [00:39<00:00, 23.76it/s]


Epoch 10: Train Loss 0.1483, Train Acc 0.9475, Train AUC 0.9926


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 52.59it/s]

Epoch 10: Val Loss 0.3296, Val Acc 0.9001, Val AUC 0.9815


In [14]:
torch.save({'model_state_dict':model.state_dict()},'resnet18_fourier_loss.pth')

In [15]:
model = train_fourier_aux('densenet121', train_loader, val_loader)

model.safetensors:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

Epoch 1/10 [Train]: 100%|██████████| 938/938 [01:42<00:00,  9.15it/s]


Epoch 1: Train Loss 1.1055, Train Acc 0.3553, Train AUC 0.5301


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:09<00:00, 24.47it/s]


Epoch 1: Val Loss 1.0395, Val Acc 0.4352, Val AUC 0.6331


...

Epoch 9: Val Loss 0.2158, Val Acc 0.9193, Val AUC 0.9845


Epoch 10/10 [Train]: 100%|██████████| 938/938 [01:42<00:00,  9.15it/s]


Epoch 10: Train Loss 0.1745, Train Acc 0.9383, Train AUC 0.9897


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:09<00:00, 24.56it/s]

Epoch 10: Val Loss 0.2250, Val Acc 0.9192, Val AUC 0.9853


In [16]:
torch.save({'model_state_dict':model.state_dict()},'densenet121_fourier_loss.pth')

In [20]:
model = train_fourier_aux('mobilenetv2_100', train_loader, val_loader)

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Epoch 1/10 [Train]: 100%|██████████| 938/938 [00:52<00:00, 18.04it/s]


Epoch 1: Train Loss 1.1048, Train Acc 0.4352, Train AUC 0.6219


Epoch 1/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 53.03it/s]


Epoch 1: Val Loss 0.9180, Val Acc 0.5400, Val AUC 0.7357


...

Epoch 9: Val Loss 0.2819, Val Acc 0.9003, Val AUC 0.9766


Epoch 10/10 [Train]: 100%|██████████| 938/938 [00:52<00:00, 17.83it/s]


Epoch 10: Train Loss 0.2041, Train Acc 0.9257, Train AUC 0.9869


Epoch 10/10 [Val]: 100%|██████████| 235/235 [00:04<00:00, 52.04it/s]

Epoch 10: Val Loss 0.3424, Val Acc 0.8879, Val AUC 0.9768


In [21]:
torch.save({'model_state_dict':model.state_dict()},'mobilenetv2_fourier_loss.pth')